## BiLSTM with ARAVEC
In this notebook, we will walk you through the process of reproducing the BiLSTM with ARAVEC baseline for the Multi-label classification task.

### Loading Required Modules
First off, we start by loading the needed python libraries.

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score, jaccard_score

from embed_classer import embed
from utils import normalize_text, create_model

### Loading Data
Using pandas, we can load and inspect the training, validation, and testing datasets as follows:

In [ ]:
df_train = pd.read_csv('../Data/train.tsv', sep="\t")
df_dev = pd.read_csv("../Data/validation.tsv", sep="\t")
df_test = pd.read_csv("../Data/test_unlabaled.tsv", sep="\t")

Below we list the 5 first entries in the training data.

In [ ]:
df_train.head()

And the 5 first entries in the development data.

In [ ]:
df_dev.head()

And last but not least, the first 5 entries in the test data.

In [ ]:
df_test.head()

### Model Preparation
We start by setting the randomisation seed and the maximum sentence length:

In [ ]:
seed_val = 32
tf.random.set_seed(seed_val)
max_sentence_len = 256

# Load Aravec 
embedd_path = '../models/full_uni_sg_300_twitter.mdl'
embedd_size = 300

# Save Model
model_path = '../models/bi_lstm.best.hdf5'

Next we load our embedding model:

In [ ]:
embedder = embed(embedd_path)

### Model Training
First we perpare the inputs and outputs to be fed to the model during training:

In [ ]:
X_train = np.array([normalize_text(text) for text in df_train.Article.values])
Y_train = df_train[df_train.columns[1:]].values

X_dev = np.array([normalize_text(text) for text in df_dev.Article.values])
Y_dev = df_dev[df_dev.columns[1:]].values

X_train = embedder.embed_batch(X_train, max_sentence_len)
X_dev = embedder.embed_batch(X_dev, max_sentence_len)

Then we define the callbacks for EarlyStopping and ModelCheckpoint

In [ ]:
earlystopping_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

Next we fit the data:

In [ ]:
model = create_model(max_sentence_len, embedd_size)
history = model.fit(X_train,
          Y_train,
          epochs=5
          batch_size=32,
          validation_data = (X_dev, Y_dev),
          callbacks=[checkpoint_callback, earlystopping_callback])

We calculate the F1-Score and Jaccard score for the development set as follows:

In [ ]:
preds = model.predict(X_dev) > 0.5

print("F1 macro:   {}".format(round(f1_score(Y_dev, preds, average="macro"), 3)))
print("F1 micro:   {}".format(round(f1_score(Y_dev, preds, average="micro"), 3)))
print("F1 samples: {}".format(round(f1_score(Y_dev, preds, average="samples"), 3)))

In [ ]:
jaccard_samples = jaccard_score(Y_dev, preds, average="samples")
jaccard_macro = jaccard_score(Y_dev, preds, average="macro")
jaccard_micro = jaccard_score(Y_dev, preds, average="micro")

print("Jaccard Macro Score:         {}".format(round(jaccard_macro, 3)))
print("Jaccard Micro Score:         {}".format(round(jaccard_micro, 3)))
print("Jaccard samples Score:       {}".format(round(jaccard_samples, 3)))

### Submission Preperation
We perpare the features for each testset instance as follows:

In [ ]:
X_test = np.array([normalize_text(text) for text in df_test.Article.values])
X_test = embedder.embed_batch(X_test, max_sentence_len)
preds = model.predict(X_test) > 0.5

Next we save the predictions into a submission file

In [ ]:
df = pd.DataFrame(data=preds, index=None, columns=None, dtype=int)
df.to_csv("../Data/outputs/answer.tsv", header=False, index=False, sep="\t")